In [1]:
import csv
import matplotlib.pyplot as plt
import numpy as np
import os
from wettbewerb import load_references, get_3montages, get_6montages
import mne
from scipy import signal as sig
import ruptures as rpt
import pywt

In [2]:
training_folder = "../test"

In [3]:
ids, channels, data, sampling_frequencies, reference_systems, eeg_labels = load_references(training_folder) 
# Importiere EKG-Dateien, zugehörige Diagnose, Sampling-Frequenz (Hz) und Name (meist fs=256 Hz)

1000	 Dateien wurden geladen.


In [4]:
# Decompose the wave
wavelet = 'db4'
dataset_montage_line_length_array = np.zeros((len(ids),15))
for i,_id in enumerate(ids):
    montage, montage_data, is_missing = get_3montages(channels[i], data[i])
    montage_line_length_array = np.zeros((15))
    for j, signal_name in enumerate(montage):
        ca4, cd4, cd3, cd2, cd1 = pywt.wavedec(montage_data[j], wavelet, level=4)
        montage_line_length_array[(5*j)] = np.sum(np.abs(np.diff(ca4)))/len(ca4)
        montage_line_length_array[(5*j)+1] = np.sum(np.abs(np.diff(cd4)))/len(cd4)
        montage_line_length_array[(5*j)+2] = np.sum(np.abs(np.diff(cd3)))/len(cd3)
        montage_line_length_array[(5*j)+3] = np.sum(np.abs(np.diff(cd2)))/len(cd2)
        montage_line_length_array[(5*j)+4] = np.sum(np.abs(np.diff(cd1)))/len(cd1)
    dataset_montage_line_length_array[i] = montage_line_length_array

In [5]:
dataset_montage_line_length_array

array([[6.87096789e-05, 2.44867114e-05, 1.13180397e-05, ...,
        9.29530494e-06, 6.26150107e-06, 2.42339434e-05],
       [7.68369260e-05, 2.71320109e-05, 1.25930928e-05, ...,
        1.04708655e-05, 6.29303622e-06, 2.33241354e-05],
       [7.49150167e-05, 2.63315376e-05, 1.22092122e-05, ...,
        9.98153679e-06, 6.44460660e-06, 2.44918581e-05],
       ...,
       [6.82400683e-05, 7.24855439e-05, 4.33980325e-05, ...,
        2.99905104e-06, 2.52739698e-06, 1.06150371e-06],
       [1.45126718e-05, 6.11035196e-06, 2.70857945e-06, ...,
        2.08439957e-06, 1.90950991e-06, 9.58611141e-07],
       [1.24998557e-05, 5.11079692e-06, 2.13532587e-06, ...,
        1.27991571e-06, 1.26903199e-06, 9.24636941e-07]])

In [24]:
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Define the number of folds for cross-validation
n_folds = 5

# Initialize an empty list to store cross-validation scores
scores = []
features = dataset_montage_line_length_array
labels = np.array(eeg_labels, dtype=int)[:,0]
# Perform cross-validation
kf = KFold(n_folds, shuffle=True, random_state=42)
for train_index, test_index in kf.split(features):
    # Extract low-dimensional features for training and testing data
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = labels[train_index], labels[test_index]

    # Train the random forest classifier on the training data
    rf_classifier = RandomForestClassifier()
    rf_classifier.fit(X_train, y_train)

    # Make predictions on the testing data
    predictions = rf_classifier.predict(X_test)

    # Calculate the accuracy score for the current fold
    accuracy = accuracy_score(y_test, predictions)
    scores.append(accuracy)

# Calculate the average cross-validation score
average_score = np.mean(scores)
print('Average cross-validation accuracy:', average_score)
print(scores)

Average cross-validation accuracy: 0.834
[0.805, 0.865, 0.84, 0.82, 0.84]
